In [1]:
import collections
import json
import re
import string

import nltk
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from gensim.parsing.preprocessing import (preprocess_string, remove_stopwords,
                                          strip_punctuation, strip_tags)
from nltk.corpus import stopwords
from nltk.util import ngrams  # function for making ngrams
from numpy import asarray, save, savez_compressed
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
import torch

In [2]:
mpstDF= pd.read_csv("mpst.csv")
mpstDF


,imdb_id,title,plot_synopsis,tags,split,synopsis_source
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",train,imdb
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,train,imdb
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,test,imdb
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",train,imdb
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",val,imdb
...,...,...,...,...,...,...
14823,tt0219952,Lucky Numbers,"In 1988 Russ Richards (John Travolta), the wea...","comedy, murder",test,wikipedia
14824,tt1371159,Iron Man 2,"In Russia, the media covers Tony Stark's discl...","good versus evil, violence",train,wikipedia
14825,tt0063443,Play Dirty,During the North African Campaign in World War...,anti war,train,wikipedia
14826,tt0039464,High Wall,Steven Kenet catches his unfaithful wife in th...,murder,test,wikipedia


In [3]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can't", "can not", phrase)
    phrase = re.sub(r"couldn't", "could not", phrase)
    phrase = re.sub(r"wouldn't", "would not", phrase)
    phrase = re.sub(r"shouldn't", "should not", phrase)
    phrase = re.sub(r"don't", "do not", phrase)
    phrase = re.sub(r"doesn't", "does not", phrase)
    phrase = re.sub(r"haven't", "have not", phrase)
    phrase = re.sub(r"hasn't", "has not", phrase)
    phrase = re.sub(r"ain't", "not", phrase)
    phrase = re.sub(r"hadn't", "had not", phrase)
    phrase = re.sub(r"didn't", "did not", phrase)
    phrase = re.sub(r"wasn't", "was not", phrase)
    phrase = re.sub(r"aren't", "are not", phrase)
    phrase = re.sub(r"isn't", "is not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

# stop_words = stopwords.words('english')


In [4]:
mpstDF_processsed=mpstDF.copy()
# Type 1: Decontracted Text, The puncutation and stop words are still there
mpstDF_processsed["processed_synopsis_t1"]=mpstDF_processsed["plot_synopsis"].apply(lambda x: decontracted(" ".join(preprocess_string(x, [lambda x: x.lower(), strip_tags]))))
# Type 2 Decontracted Text Stop Words Removed
mpstDF_processsed["processed_synopsis_t2"]=mpstDF_processsed["plot_synopsis"].apply(lambda x: decontracted(" ".join(preprocess_string(x, [lambda x: x.lower(), strip_tags,remove_stopwords]))))

In [5]:
# from transformers import XLNetConfig, XLNetModel, XLNetTokenizer
from transformers import AutoModel,AutoConfig,AutoTokenizer


In [13]:
# xlnConfig= XLNetConfig()
# xlnModel = XLNetModel(xlnConfig)
from transformers import AutoModel,AutoConfig,AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('xlnet-base-cased')
# model = AutoModel.from_config(config)
model = AutoModel.from_pretrained('xlnet-base-cased')

In [12]:
def get_Encodings(text,tokenizer=tokenizer,model=model, verbose=False):
    if verbose:
        print("Text:")
        print(text[:20] + "....")
    text_encoding_tensor=torch.tensor([tokenizer.encode(text)])
    if verbose:
        print("text_encoding_tensor:")
        print(text_encoding_tensor)
        print("shape:")
        print(text_encoding_tensor.shape)
    attention_mask_tensor= torch.tensor([[1]*text_encoding_tensor.shape[1]])
    if verbose:
        print("attention_mask_tensor:")
        print(attention_mask_tensor)
        print("shape:")
        print(attention_mask_tensor.shape)

    with torch.no_grad():
        outputs = model(text_encoding_tensor, attention_mask=attention_mask_tensor)
        if verbose:
            print("outputs:")
            print(outputs)
            print("Lenght of outputs",len(outputs))
            print("outputs[0]:")
            print(outputs[0])
            print("outputs[0].shape:")
            print(outputs[0].shape)
            print("outputs[1]:")
            print(outputs[1])
            print("Length Ooutputstput[1]:")
            print(len(outputs[1]))
            print("Sample from Output[1], first hidden layer:")
            print(outputs[1][0])
            print("Sample shape, first hidden layer")
            
    if verbose:
        print("getting the last tensor for XLNet")
        print(outputs[0].squeeze()[-1])
    return outputs[0].squeeze()[-1]

In [25]:
def getXLNetEmbeddings(df,column,verbose=False):
    embeddings = np.array(torch.tensor([np.array(get_Encodings(x,verbose=verbose)) for x in df[column]]))
    return embeddings

In [21]:
# THIS IS NORMAL TEST FUNCTION CREATED AND LATER MNERGED WITH THE getXLNetEmbeddings METHOD
# def getXLNetEmbeddings_testMode(df,column,verbose=False):
#     embeddings = np.array(torch.tensor([np.array(get_Encodings(x,verbose=verbose)) for x in df[column]]))
#     return embeddings

**Testing verbose mode for one Input**

In [15]:
sample_text="In May 1980, a Cuban man named Tony Montana (Al Pacino) claims asylum, in Florida, USA, and is in search of the \"American Dream\" after departing Cuba in the Mariel boatlift of 1980. When questioned by three tough-talking INS officials, they notice a tattoo on Tony's left arm of a black heart with a pitchfork through it, which identifies him as a hitman, and detain him in a camp called 'Freedomtown' with other Cubans, including Tony's best friend and former Cuban Army buddy Manolo \"Manny Ray\" Ribiera (Steven Bauer), under the local I-95 expressway while the government evaluates their visa petitions.After 30 days of governmental dithering and camp rumors, Manny receives an offer from the Cuban Mafia which he quickly relays to Tony. If they kill Emilio Rebenga (Roberto Contreras) a former aide to Fidel Castro who is now detained in Freedomtown, they will receive green cards. Tony agrees, and kills Rebenga during a riot at Freedomtown."
test_output=get_Encodings(sample_text,verbose=True)
print(test_output)

Text:
In May 1980, a Cuban....
text_encoding_tensor:
tensor([[   67,   428,  1910,    19,    24,  5907,   326,   812,  3342,  9662,
            17,    10,  3869, 12012,  2844,    11,  1897, 11003,    19,    25,
          1442,    19,  3224,    19,    21,    27,    25,   987,    20,    18,
            17,    12,  2730,  7684,    12,    99, 21077,  4310,    25,    18,
          9052,   368,  2694,  7358,    20,  1910,     9,   311,  6242,    37,
           139,  2434,    13, 14877,    56,    35,  6522,   361,    19,    63,
          2588,    24, 10784,    31,  3342,    26,    23,   263,  1860,    20,
            24,   710,   758,    33,    24,  4811,  1383,   267,   135,    36,
            19,    59, 17142,   103,    34,    24,   645,   249,    19,    21,
         24417,   103,    25,    24,  1900,   271,    17,    26,  8060,  5173,
          2391,    26,    33,    86,  5907,    23,    19,   208,  3342,    26,
            23,   252,  1233,    21,   317,  5907,  1422, 20555,  1341,  8292,

Now testing the helper functions for input from a dataframe to see if we get the correct desired output for our work

In [19]:
sampleDF=mpstDF_processsed[["processed_synopsis_t1","processed_synopsis_t2"]].head(3)
sampleDF["processed_synopsis_t1"]=sampleDF["processed_synopsis_t1"].apply(lambda x : x[:1000])
sampleDF["processed_synopsis_t2"]=sampleDF["processed_synopsis_t2"].apply(lambda x : x[:1000])
display(sampleDF)

,processed_synopsis_t1,processed_synopsis_t2
0,note: this synopsis is for the orginal italian...,note: synopsis orginal italian release segment...
1,"two thousand years ago, nhagruul the foul, a s...","thousand years ago, nhagruul foul, sorcerer re..."
2,"matuschek is, a gift store in budapest, is the...","matuschek is, gift store budapest, workplace a..."


In [27]:
# sample_embeddings=getXLNetEmbeddings_testMode(sampleDF,"processed_synopsis_t1",verbose=True)
sample_embeddings=getXLNetEmbeddings(sampleDF,"processed_synopsis_t1",verbose=True)
print("sample_embeddings.shape",sample_embeddings.shape)
print(sample_embeddings)


Text:
note: this synopsis ....
text_encoding_tensor:
tensor([[ 2039,    60,    52,    17, 12681, 24243,    27,    28,    18,    49,
          5043,   212,    36,   212,   884,   947,    33,    18, 11860,    25,
            52,  1028,   374,     9,  7157,   590,    17,  4597,   368,  1372,
         17945,   139,  6941, 14753,    20,    18, 10255,  2460,    88,    21,
            18, 19707,   318,    34,    18,   116,  9494,  3462,    20,  1819,
            26,     9,   305,  2289,  4044,   117,    17,    10,    98, 23808,
           529, 23445,  2151,    11,    27,    48,  5320,    19,   227,    13,
         18736,    17, 21605,    23,   884,   547,    13, 11163,    61,  4094,
            22,    62, 16707,    19,  9464,  3246,    99,    48,  2060,    78,
            90,    85,  1262,  2298,    39,  4238,    37,    24,   461,    20,
          4572,   983,  1624,     9,    18, 18380,   802,  2399,   666,    34,
         23675,    19,    62,  2002,    13,   450,  6863,    61,    51,  1050,

HERE: make sure that the output is (3, 768). Where 3 is the number of texts we sent to the model and 768 is the output of embedding length for the model chosen.

Now We run the model for each text we have in our Dataset 

In [ ]:
#For Type 1 Embeddings
xlnet_embeddings_t1=getXLNetEmbeddings(mpstDF_processsed,"processed_synopsis_t1")

In [ ]:
print("Shape: ",xlnet_embeddings_t1.shape)
print("XL Embedding for Type 1")
print(xlnet_embeddings_t1)

In [ ]:
np.savez("xl_embeddings_type1.npz",xlnet_embeddings_t1)

In [ ]:
#For Type 2 Embeddings
xlnet_embeddings_t2=getXLNetEmbeddings(mpstDF_processsed,"processed_synopsis_t2")


In [ ]:
print("Shape: ",xlnet_embeddings_t2.shape)
print("Embedding")
print(xlnet_embeddings_t2)

In [ ]:
np.savez("xl_embeddings_type1.npz",xlnet_embeddings_t2)

Saved the embeddings in different files
Now saving them in the same file

In [ ]:
np.savez("xl_embeddings.npz",t1=xlnet_embeddings_t1,t2=xlnet_embeddings_t2)

In [ ]:
em_check=np.load("xl_embeddings.npz")
print("t1")
print(em_check["t1"])
print(em_check["t1"].shape)
print("t2")
print(em_check["t2"])
print(em_check["t2"].shape)
